In [160]:
%matplotlib notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math
x0=1


In [180]:
# Modello di Ising tramite metropolis

def energia(Jconst, nx, ny, spins):
    if(isinstance(spins[0], np.ndarray)):
        spins = np.concatenate(spins).flat
    ene=0.
    icount=0
    for j in range(0, ny, 1):
        for i in range(0, nx, 1):
            pv_x,pv_y,pv_count = 0, 0, 0
            if i<nx-1:
                pv_x=i+1
            else:
                pv_x=0
            pv_y=j
            pv_count=pv_y*nx+pv_x
            ene+=-0.5*Jconst*spins[icount]*spins[pv_count]
            
            if i>0:
                pv_x=i-1
            else:
                pv_x=nx-1
            pv_y=j
            pv_count=pv_y*nx+pv_x
            ene+=-0.5*Jconst*spins[icount]*spins[pv_count]
            
            if j<ny-1:
                pv_y=j+1
            else:
                pv_y=0
            pv_x=i
            pv_count=pv_y*nx+pv_x
            ene+=-0.5*Jconst*spins[icount]*spins[pv_count]
            
            if j>0:
                pv_y=j-1
            else:
                pv_y=ny-1
            pv_x=i
            pv_count=pv_y*nx+pv_x
            ene+=-0.5*Jconst*spins[icount]*spins[pv_count]
            
            icount+=1
    return ene

def numero_random( ):
    a=16807
    c=0
    m=2147483647
    global x0
    x1=(a*x0+c)%m
    r=float(x1)/float(m)
    x0 = x1
    return r

def genera_configurazione(nx, ny, spins):
    for j in range(0, ny, 1):
        for i in range(0, nx, 1):
            r =numero_random()
            if (r<0.5):
                spins[j][i]=-1
            else:
                spins[j][i]=1
            
    return spins

In [256]:
def cambia_configurazione( Jconst, nx,  ny,  spins):
    spins = np.concatenate(spins).flat
    spins1=spins.copy()
    
    ene=0
    
    r=numero_random()
    icount=math.floor(r*(nx*ny))
    spins1[icount]*=-1
    j=math.floor(icount/nx)
    i=icount-j*nx
    
    
    if(i<nx-1):
        pv_x=i+1
    else:
        pv_x=0
    
    pv_y=j
    pv_count=pv_y*nx+pv_x
    ene+=-Jconst*spins1[icount]*spins1[pv_count]
    ene-=-Jconst*spins[icount]*spins[pv_count]
    
    if (i>0):
        pv_x=i-1
    else:
        pv_x=nx-1
    
    pv_y=j
    pv_count=pv_y*nx+pv_x
    ene+=-Jconst*spins1[icount]*spins1[pv_count]
    ene-=-Jconst*spins[icount]*spins[pv_count]
    
    if (j<ny-1):
        pv_y=j+1
    else:
        pv_y=0
    pv_x=i
    pv_count=pv_y*nx+pv_x
    ene+=-Jconst*spins1[icount]*spins1[pv_count]
    ene-=-Jconst*spins[icount]*spins[pv_count]
    
    if  j > 0 :
        pv_y=j-1
    else:
        pv_y=ny-1
    
    pv_x=i
    pv_count=pv_y*nx+pv_x
    ene+=-Jconst*spins1[icount]*spins1[pv_count]
    ene-=-Jconst*spins[icount]*spins[pv_count]
    
    return ene, np.array(spins1).reshape(nx, ny)
    

def magnetizzazione(nx, ny, spins):
    icount=0
    ma=0
    spins =  np.concatenate(spins).flat
    for j in range(0, ny):
        for i in range(0, nx):
            ma+=spins[icount]
            icount+=1
    return ma

In [257]:
def metropolis(spins, steps, nx, ny, J, kbT, energy):
    spins1 = spins.copy()
    d_spins = np.zeros(steps)
    d_energy = np.zeros(steps)
    for s in range(0, steps):
        d_ene, spins1 = cambia_configurazione(J, nx, ny, spins)
        if (d_ene >0)*(numero_random() < np.exp(-kbT * d_ene)):
            spins = spin s1
            energy += d_ene
        elif d_ene <= 0 :
            spins = spins1
            energy += d_ene
        d_energy[s] = energy
        d_spins[s] = spins.sum()
    return d_energy, d_spins, spins

In [264]:
nsteps = 10**8
def main():
    nx,ny = 50, 50 # 20, 20 # 50, 50
    J, kbT = 1, 1 # kbT in [0, 3]
    ene_ave=0
    ene_var=0
    ma_ave=0
    ma_var=0
    masq_ave=0
    enesq_ave=0
    
    ratio,r = 0, 0
    metodo = 1
    
    ene = np.zeros(nsteps)
    ma = np.zeros(nsteps)
    
    spins = np.zeros((nx,ny), dtype = float)
    spins1 = np.zeros((nx,ny), dtype = float)
    
    spins = genera_configurazione(nx, ny, spins)
    dE, dSpins, spins = metropolis(spins, nsteps, nx, ny, J, kbT, energia(J, nx, ny, spins))
    return dE, dSpins, nx, ny, spins
de, dspi, nx, ny, spins = main()

In [1]:
fig, axs = plt.subplots(1,2)
t = np.linspace(0,nsteps, nsteps)
print(spins)
axs[0].plot(de/(nx*ny))
axs[1].plot(dspi/(nx*ny))
plt.show()

NameError: name 'plt' is not defined

In [266]:
axs[0].imshow(spins)

In [249]:
spins.sum()

1994.0

In [250]:
spins.sum()/2500

0.7976